In [1]:
import pandas as pd 
import numpy as np 
import sqlite3

## Notes

In [12]:
#STRUCTURE
# users table - added
# accounts table - added
# account_types table - added
# beneficiaries
# loan_types _?
# loans _ ?
# loan_applications _ ?
# card_types - added
# cards - added
# branches
# employees
# deposits
# transaction_types
# transactions table  


In [ ]:
# view table is needed to check how much left from loan (next version)

In [13]:
# In a banking scenario, a "loans" table would typically store information about loans issued by the bank to its customers. The specific columns you would need in this table depend on the details you want to track. Here's a list of common columns you might include in a "loans" table:

# loan_id: A unique identifier for each loan.
# customer_id: The ID of the customer who received the loan.
# loan_amount: The amount of money lent for the loan.
# interest_rate: The interest rate for the loan.
# loan_type: The type of loan (e.g., personal, mortgage, business).
# start_date: The date when the loan was issued.
# end_date: The expected or actual date when the loan will be fully repaid.
# term_months: The loan term in months.
# status: The status of the loan (e.g., active, paid off, overdue).
# collateral: Information about any collateral provided for the loan.
# monthly_payment: The monthly payment amount.
# remaining_balance: The remaining balance of the loan.
# early_payment_penalty: Penalty for early repayment (if applicable).
# late_payment_penalty: Penalty for late payments (if applicable).
# notes: Any additional notes or comments related to the loan.

## CODE

In [5]:
conn = sqlite3.connect('db.db')

def check_table(table_name):
    try:
        q = f"select * from {table_name}"
        print("query created")
    except:
        print('query can not be created')

    try:
        output = conn.execute(q).fetchall()
        print("output received")
    except sqlite3.Error as err:
        print(err)
    print("table data")
    conn.close()
    return output

def show_tables (connection):
    try:
        q = """ select * FROM sqlite_master 
                WHERE type='table'"""
    except sqlite3.Error as err:
        print(err)
    conn.close()
    conn.execute(q).fetchall()

def run_query(query, db):
    q = query
    conn = sqlite3.connect(f'{db}.db')
    output = conn.execute(q).fetchall()
    return output

In [36]:
create_users_table = """CREATE TABLE IF NOT EXISTS users (
    id INTEGER Primary Key
    ,first_name TEXT
    ,last_name TEXT
    ,phone_number TEXT
    ,pin_code TEXT CHECK( LENGHT(pin_code) = 6)
    ,email TEXT
    ,created_at TIMESTAMP DEFAULT CURRENT_TIME
--    ,udpated_at TIMESTAMP DEFAULT CURRENT_TIME
)"""

creat_account_types_table= """
CREATE TABLE IF NOT EXISTS account_types(
    id INTEGER PRIMARY KEY
    ,account_name TEXT
    ,locale TEXT CHECK (LENGTH(locale) <= 3)
)
"""

create_accounts_table = """ 
CREATE TABLE IF NOT EXISTS accounts(
    id INTEGER Primary Key
    ,account_type INTEGER
    ,user_id INTEGER
    ,FOREIGN KEY(user_id) REFERENCES users(id)
    ,FOREIGN KEY(account_type) REFERENCES account_types(id)
)
"""

create_card_types_table = """ 
CREATE TABLE IF NOT EXISTS card_types(
    id INTEGER Primary Key
    ,card_name TEXT 
    ,locale TEXT CHECK ( LENGTH (locale) <= 3)
)

"""

create_cards_table = """
CREATE TABLE IF NOT EXISTS cards(
    id INTEGER Primary Key
    , card_number INTEGER CHECK( LENGTH (card_number) = 16)
    , card_type INTEGER
    , account INTEGER
    , FOREIGN KEY (card_type) REFERENCES card_types(id)
    , FOREIGN KEY (account) REFERENCES accounts(id)
)
"""

# create_loans = """ 
# CREATE TABLE IF NOT EXISTS loan_types (
#     id INTEGER Primary Key
#     ,loan_name TEXT 
#     ,grace_period_days INTEGER CHECK (LENGTH (grace_period_days) <= 3)
# )
# """

create_transaction_types_table = """
CREATE TABLE IF NOT EXISTS transaction_types(
    id INTEGER PRIMARY KEY
    ,transaction_name TEXT 
    ,locale TEXT CHECK (LENGTH (locale) <= 3)
    -- withdrawal, payment, deposit, transfer
)
"""


#https://github.com/datasets/currency-codes/blob/master/data/codes-all.csv
create_currencies_table = """
CREATE TABLE IF NOT EXISTS currencies(
    id INTEGER Primary Key
    ,country_name
    ,currency_code TEXT CHECK( LENGTH(currency_name) <=3)
)
"""

create_fee_types_table  ="""
CREATE TABLE IF NOT EXISTS fees(
    id INTEGER Primary Key
    ,fee_name TEXT
    ,description TEXT
)
"""



create_transactions_table = """
CREATE TABLE IF NOT EXISTS transactions(
    id INTEGER Primary Key
    ,by_user INTEGER
    ,amount FLOAT
    ,fee FLOAT
    ,currency TEXT
    ,reference_number INTEGER
    ,transaction_method TEXT
    -- ATM, online banking
    ,from_account_id INTEGER
    ,to_account_id INTEGER
    ,transaction_type INTEGER
    ,transaction_date TIMESTAMP DEFAULT CURRENT_TIME
    ,status TEXT 
    ,note TEXT CHECK ( LENGTH(note) < 256 )

    , FOREIGN KEY (user_id) REFERENCES users(id)
    , FOREIGN KEY (currency) REFERENCES currencies(currency_code)
    , FOREIGN KEY (from_account_id) REFERENCES accounts(id)
    , FOREIGN KEY (to_account_id) REFERENCES accounts(id)
    , FOREIGN KEY (transaction_type) REFRENCRES transaction_types(id)
)
"""






# CREATE TABLE accounts (
#     id INTEGER Primary Key,
#     user_id INTEGER,
#     account_type INTEGER,
#     balance REAL,
#     FOREIGN KEY (user_id) REFERENCES users(id),
#     FOREIGN KEY (account_type) REFERENCES account_types(id)
# )

# CREATE TABLE account_types (
#     id INTEGER Primary Key,
#     account_name TEXT
# )
# """

In [37]:
tables = [create_users_table,creat_account_types_table,create_accounts_table,create_card_types_table,create_cards_table,create_transaction_types_table,create_currencies_table,
create_fee_types_table,create_transactions_table]

In [38]:
for x in tables:
    conn.execute(x).fetchall()

OperationalError: near "account": syntax error

In [35]:
conn.execute(tables[2])

OperationalError: near "user_id": syntax error

In [22]:
q = """ select * FROM sqlite_master 
        WHERE type='table'"""

In [23]:
conn.execute(q).fetchall()

[('table',
  'users',
  'users',
  2,
  'CREATE TABLE users (\n    id INTEGER Primary Key,\n    first_name TEXT ,\n    last_name TEXT,\n    phone_number TEXT,\n    pin_code TEXT,\n    email TEXT,\n    created_at TIMESTAMP DEFAULT CURRENT_TIME,\n    udpated_at TIMESTAMP DEFAULT CURRENT_TIME\n)')]

## Next Version Parts

In [4]:
# TO change udpated_at on each modification for next version
query = """CREATE TRIGGER update_your_table_name 
AFTER UPDATE ON your_table_name
FOR EACH ROW
BEGIN
    UPDATE your_table_name
    SET updated_at = CURRENT_TIMESTAMP
    WHERE id = OLD.id;
END;
"""